In [1]:
# ------------------------------ Packages & Files ------------------------------
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import ipywidgets as widgets
from IPython.display import display, HTML

from ppa_analysis import user_inputs, advanced_settings, hybrid, import_data,\
    bill_calc, battery, firming_contracts

INFO: Using Python-MIP package version 1.16rc0


In [2]:
# ------------------------------ Initialise the input collector ----------------
input_collector = user_inputs.launch_input_collector()

C:\Users\nick\Documents\GitHub\ppa_analysis
data_caches\yearly_data_files


Dropdown(description='Year:', options=('2020', '2021'), value='2020')

Dropdown(description='Generator region:', options=('QLD1', 'NSW1', 'VIC1', 'SA1', 'TAS1'), value='QLD1')

Dropdown(description='Load region:', options=('QLD1', 'NSW1', 'VIC1', 'SA1', 'TAS1'), value='QLD1')

Dropdown(description='Load data file:', options=('(18) Hospital NQ.csv', '(20) Sawmill WB.csv', '(22) Hospital…

...

BoundedFloatText(value=1.0, description='Matching percentile:')

FloatText(value=300.0, description='Exposure upper bound ($/MW/h):')

FloatText(value=20.0, description='Exposure lower bound ($/MW/h):')

Dropdown(description='Time series interval:', options=('60',), value='60')

Dropdown(description='Generator data set:', options=('GenCost 2018 Low',), value='GenCost 2018 Low')

In [3]:
generator_data_editor = user_inputs.launch_generator_data_editor(input_collector)

Output()

In [4]:
# ------------------- Unpack LCOE Inputs -------------------------

# Calculate LCOE from user inputs/predetermined values
# Function takes in the  generator LCOE info dictionary, and calculates LCOE
# for only one generator with each call.
# Returns LCOE value in $/MW
def calculate_lcoe(
    generator_info:dict[str:object]        
) -> float:
    
    # Baseline assumptions:
    lifetime_years = 25
    discount_rate = 0.07         # AEMC uses 6-8.5% for all technologies

    capital_cost = generator_info['capital'].value * generator_info['capacity'].value
    numerator, denominator = 0, 0
    for year in range(1,lifetime_years+1):
        kwh_in_year_n = generator_info['capacity_factor'].value*generator_info['capacity'].value*(365*24)   # Note: this doesn't currently account for leap years!
        numerator += (generator_info['fixed_om'].value * generator_info['capacity'].value\
                      + generator_info['variable_om'].value * kwh_in_year_n) / \
                        ((1 + discount_rate) ** year)
        denominator += (kwh_in_year_n) / ((1 + discount_rate) ** year)
    numerator += capital_cost

    return (numerator / denominator)  * 1000


# ----- Fetch inputs and set up info_dict data to pass to later functions:
def get_all_lcoes(
        generator_data_editor:dict[str:dict[str:object]]
) -> dict[str:float]:
    all_generator_lcoes = {}
    for gen, gen_info in generator_data_editor.items():
        if gen != 'out':
            gen_lcoe = calculate_lcoe(gen_info)
            all_generator_lcoes[gen] = gen_lcoe
    
    return all_generator_lcoes


In [5]:
# Maybe this function is where the load gets pulled in, all the other data collecting
# functions get called etc

def collect_and_combine_data(
        input_collector:dict
) -> pd.DataFrame:    
    # ----------------------------- Unpack user input ------------------------------
    year_to_load_from_cache = input_collector['year'].value
    year_to_load = int(year_to_load_from_cache)
    GENERATOR_REGION = input_collector['generator_region'].value
    LOAD_REGION = input_collector['load_region'].value
    generators = list(input_collector['generators'].value)

    # ------------------- Get Load Data --------------------
    # if using preset data, use these hard coded values:
    LOAD_DATA_DIR = 'data_caches/c_and_i_customer_loads'
    load_filename = input_collector['load_data_file'].value
    filepath = LOAD_DATA_DIR + '/' + load_filename
    LOAD_DATETIME_COL_NAME = 'TS'
    LOAD_COL_NAME = 'Load'
    DAY_FIRST = True

    # Units are definitely a question.
    load_data, start_date, end_date = import_data.get_load_data(filepath, LOAD_DATETIME_COL_NAME, LOAD_COL_NAME, DAY_FIRST)
    load_data = load_data / 1000    # convert to MWh
    load_data = load_data[
        (load_data.index >= f'{year_to_load}-01-01 00:00:00') & 
        (load_data.index < f'{year_to_load+1}-01-01 00:00:00')
    ]

    # else: TODO add another option here if users want to load in their own data

    # ----------------------------- Get Generation Data ----------------------------
    gen_data_file = (
        advanced_settings.YEARLY_DATA_CACHE / 
        f'gen_data_{year_to_load_from_cache}.parquet'
    )
    gen_data = import_data.get_preprocessed_gen_data(
        gen_data_file, [GENERATOR_REGION]
    )
    gen_data = gen_data[generators]

    # --------------------------- Get Emissions Data -------------------------------
    emissions_data_file = (
        advanced_settings.YEARLY_DATA_CACHE / 
        f'emissions_data_{year_to_load_from_cache}.parquet'
    )
    emissions_intensity = import_data.get_preprocessed_avg_intensity_emissions_data(
        emissions_data_file, [LOAD_REGION, GENERATOR_REGION]
    )

    # ------------------------ Get Wholesale Price Data ----------------------------
    price_data_file = (
        advanced_settings.YEARLY_DATA_CACHE / 
        f'price_data_{year_to_load_from_cache}.parquet'
    )
    price_data = import_data.get_preprocessed_price_data(
        price_data_file, [LOAD_REGION, GENERATOR_REGION]
    )

    combined_data = pd.concat([load_data, gen_data, price_data, emissions_intensity], axis='columns')

    FIRMING_CONTRACT_TYPE = input_collector['firming_contract_type'].value
    EXPOSURE_BOUND_UPPER = input_collector['exposure_upper_bound'].value
    EXPOSURE_BOUND_LOWER = input_collector['exposure_lower_bound'].value
    RETAIL_TARIFF_DETAILS = {}

    # Add the firming details:
    combined_data = firming_contracts.choose_firming_type(
        FIRMING_CONTRACT_TYPE, combined_data, [LOAD_REGION], EXPOSURE_BOUND_UPPER, EXPOSURE_BOUND_LOWER, RETAIL_TARIFF_DETAILS
    )

    return combined_data

In [6]:
# At the moment: this is all assuming the use of sample load data stored with the tool

gen_info = get_all_lcoes(generator_data_editor)
combined_data = collect_and_combine_data(input_collector)

# Had an issue with first row containing a few NaN values causing problems for 
# the firming column addition. But just using dropna without checks not a safe bet
# necessarily...
combined_data = combined_data.dropna(how='any', axis='rows')

Some missing data found. Filled with zeros.



In [7]:
combined_data.head()

,Load,CSPVPS1: PHOTOVOLTAIC FLAT PANEL,COOPGWF1: WIND - ONSHORE,DDSF1: PHOTOVOLTAIC FLAT PANEL,KSP1: PHOTOVOLTAIC FLAT PANEL,RRP: QLD1,AEI: QLD1,Firming price: QLD1
DateTime,,,,,,,,
2020-01-01 01:00:00,1.375219,0.00000,78.052959,0.000000,0.0,51.169856,0.857502,51.169856
2020-01-01 02:00:00,1.368420,0.00000,80.647394,0.000000,0.0,51.729121,0.856617,51.729121
2020-01-01 03:00:00,1.380368,0.00000,69.206657,0.000000,0.0,51.434404,0.857574,51.434404
2020-01-01 04:00:00,1.292280,0.00000,59.052380,0.000000,0.0,48.489593,0.860999,48.489593
2020-01-01 05:00:00,1.363659,0.14225,57.830890,1.214922,0.0,48.364535,0.861482,48.364535


In [8]:
# Now calculate the hybrid percentage here:
contract_type = input_collector['contract_type'].value
redefine_period = input_collector['redefine_period'].value
contract_amount = input_collector['contract_amount'].value
load_region = input_collector['load_region'].value
time_series_interval = input_collector['time_series_interval'].value
matching_percentile = input_collector['matching_percentile'].value

opt_hybrid, percentages = hybrid.create_hybrid_generation(
    contract_type, 
    redefine_period,  
    contract_amount, 
    combined_data, 
    load_region, 
    gen_info, 
    time_series_interval, 
    matching_percentile
)

# TODO: add chart here to show average day of generators and hybrid?

In [11]:
settlement_period = input_collector['settlement_period'].value
load_region = input_collector['load_region'].value
strike_price = input_collector['strike_price'].value
lgc_buy_price = input_collector['lgc_buy_price'].value
lgc_sell_price = input_collector['lgc_sell_price'].value
shortfall_penalty = input_collector['shortfall_penalty'].value
guaranteed_percent = input_collector['guaranteed_percent'].value
excess_price = input_collector['excess_price'].value
indexation = input_collector['indexation'].value
index_period = input_collector['index_period'].value
floor_price = input_collector['floor_price'].value

bill = bill_calc.calculate_bill(
    volume_and_price=combined_data, 
    settlement_period=settlement_period, 
    contract_type=contract_type, 
    load_region=load_region, 
    strike_price=strike_price, 
    lgc_buy_price=lgc_buy_price, 
    lgc_sell_price=lgc_sell_price, 
    shortfall_penalty=shortfall_penalty, 
    guaranteed_percent=guaranteed_percent, 
    excess_price=excess_price, 
    indexation=indexation, 
    index_period=index_period, 
    floor_price=floor_price
)

In [12]:
bill

,PPA Value,PPA Settlement,Firming Costs,Revenue from on-sold RE,Revenue from excess LGCs,Cost of shortfall LGCs,Shortfall Payments Received,Total
DateTime,,,,,,,,
2020-12-31,1.136597e+06,367588.47652,172479.910045,-170300.202407,-25108.78771,0.0,-0.0,1.113668e+06


In [13]:
wholesale_bill = bill_calc.calculate_wholesale_bill(
    df=combined_data,
    settlement_period=settlement_period,
    load_region=load_region,
    lgc_buy_price=lgc_buy_price
)

wholesale_bill

,Wholesale Cost,LGC Cost,Total
DateTime,,,
2020-12-31,502857.133076,411682.07805,914539.211126


## Add a behind-the-meter battery:

In [14]:
battery_input_collector = user_inputs.launch_battery_input_collector()

FloatText(value=1.0, description='Rated power capacity (MW):')

FloatText(value=2.0, description='Battery size (MWh):')

In [15]:
# Battery
rated_power_capacity = battery_input_collector['rated_power_capacity'].value
size_in_mwh = battery_input_collector['size_in_mwh'].value

with_a_battery = battery.run_battery_optimisation(
    df=combined_data,
    load_col_to_use='Load', # Default - field may not even be useful long-term
    region=load_region,
    rated_power_capacity=rated_power_capacity,
    size_in_mwh=size_in_mwh
    # keeping charging and discharging efficiency as defaults for the moment...
)

InterfacingError: Gurobi environment could not be loaded, check your license.

In [ ]:
# Load Flex